<a href="https://colab.research.google.com/github/CakeNuthep/Super_AI/blob/master/LSTM/LSTM_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
import numpy as np

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data
NUM_WORDS = 20000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

In [ ]:
print(len(x_train[0]))
print(len(x_train[1]))

218
189


In [ ]:
MAX_LENGTH = 300

x_train_2D = np.zeros([25000,MAX_LENGTH],dtype=np.int32)
for i in range(x_train.shape[0]):
    if len(x_train[i]) >= MAX_LENGTH:
        x_train_2D[i,:] = x_train[i][0:MAX_LENGTH]
    else:
        x_train_2D[i,0:len(x_train[i])] = x_train[i]
        x_train_2D[i,len(x_train[i]):] = NUM_WORDS + 1

x_test_2D = np.zeros([25000,MAX_LENGTH],dtype=np.int32)
for i in range(x_test.shape[0]):
    if len(x_test[i]) >= MAX_LENGTH:
        x_test_2D[i,:] = x_test[i][0:MAX_LENGTH]
    else:
        x_test_2D[i,0:len(x_test[i])] = x_test[i]
        x_test_2D[i,len(x_test[i]):] = NUM_WORDS + 1

In [ ]:
whos

Variable                        Type          Data/Info
-------------------------------------------------------
Bidirectional                   type          <class 'tensorflow.python<...>.wrappers.Bidirectional'>
Conv1D                          type          <class 'tensorflow.python<...>rs.convolutional.Conv1D'>
Dense                           type          <class 'tensorflow.python<...>keras.layers.core.Dense'>
Dropout                         type          <class 'tensorflow.python<...>ras.layers.core.Dropout'>
Embedding                       type          <class 'tensorflow.python<...>rs.embeddings.Embedding'>
Flatten                         type          <class 'tensorflow.python<...>ras.layers.core.Flatten'>
Input                           function      <function Input at 0x7fbe74fc9e18>
LSTM                            type          <class 'tensorflow.python<...>ayers.recurrent_v2.LSTM'>
MAX_LENGTH                      int           300
MaxPool1D                       type       

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Input, Bidirectional, Conv1D, MaxPool1D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop

In [ ]:
input = Input((MAX_LENGTH,)) # 300
x = Embedding(NUM_WORDS+2, 400)(input) # remove this layer if use separate embedding such as fasttext/pythai etc. Feed 3D embedding directly to model
x = Conv1D(32,3,padding='same',activation='relu')(x)
x = MaxPool1D()(x) #(batch,150,32)

x = Bidirectional( LSTM(100, return_sequences=True) )(x) #3D
x = Bidirectional( LSTM(100, return_sequences=False) )(x) 

x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
out = Dense(2, activation='softmax')(x)

In [ ]:
model = Model(input, out)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 400)          8000800   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 32)           38432     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 150, 32)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 150, 200)          106400    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [ ]:
model.compile(
    optimizer=RMSprop(learning_rate=0.0005, momentum=0.9),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
model.fit(x_train_2D, y_train, epochs=5)

Epoch 1/5
782/782 [==============================] - 31s 39ms/step - loss: 0.5263 - accuracy: 0.7402
Epoch 2/5
782/782 [==============================] - 30s 38ms/step - loss: 0.3190 - accuracy: 0.8702
Epoch 3/5
782/782 [==============================] - 30s 39ms/step - loss: 0.2447 - accuracy: 0.9061
Epoch 4/5
782/782 [==============================] - 30s 39ms/step - loss: 0.1949 - accuracy: 0.9289
Epoch 5/5
782/782 [==============================] - 30s 39ms/step - loss: 0.1559 - accuracy: 0.9475


In [ ]:
model.evaluate(x_test_2D,  y_test, verbose=2)

782/782 - 11s - loss: 0.3278 - accuracy: 0.8758


[0.3278350532054901, 0.8758400082588196]